In [158]:
import pandas as pd

cols = ['Name', 'Kategorie', 'Bezugseinheit', 'Energie, Kalorien', 
        'Fett, total (g)', 'Kohlenhydrate, verfügbar (g)', 'Nahrungsfasern (g)',
        'Protein (g)']

names = ['name', 'category', 'units', 'calories', 'fatt',
         'carbs', 'fiber', 'protein']

foods = ((pd.read_excel('Schweizer-Nährwertdatenbank-V6.1.xlsx',
                       skiprows=2, usecols=cols))
        )

foods.columns = names

def to_lower(df, col):
    df = df.copy()
    df[col] = df[col].str.lower()
    return df
    
foods = to_lower(foods, 'name')



In [172]:
food = 'lins'
foods.loc[foods.name.str.contains(food)]

,name,category,units,calories,fatt,carbs,fiber,protein
482,"linse, ganz, gekocht (ohne zugabe von fett und...","Getreideprodukte, Hülsenfrüchte und Kartoffeln...",pro 100g essbarer Anteil,119,0.5,16.4,6.2,8.9
483,"linse, ganz, getrocknet","Getreideprodukte, Hülsenfrüchte und Kartoffeln...",pro 100g essbarer Anteil,324,1.5,44.8,17,24.4
484,"linse, geschält, gekocht (ohne zugabe von fett...","Getreideprodukte, Hülsenfrüchte und Kartoffeln...",pro 100g essbarer Anteil,139,0.6,20,4.3,11.3
485,"linse, geschält, getrocknet","Getreideprodukte, Hülsenfrüchte und Kartoffeln...",pro 100g essbarer Anteil,334,1.5,48,10.4,27


In [ ]:
meal = ['dattel, getrocknet']